### 학습목표 

- 웹 스크래핑 : 웹 사이트 상에서 위치를 지정해서 원하는 데이터를 추출하는 방법(정적 파일)
    - node walking 태그를 따라 타고 들어간다 
    - css selector를 이용해서 다이렉트로 접근 가능

- 웹 크롤링 : 자동화 봇을 이용하여 링크를 따라서 연결된 페이지를 가져와서 원하는 데이터를 추출하는 방법(동적 파일)

In [1]:
import numpy as np
import pandas as pd

%matplotlib inline

from datetime import date,datetime,timedelta
from dateutil.parser import parse

import json
import re
from glob import glob

import seaborn as sns
import folium as g

import matplotlib.pyplot as plt
from matplotlib import rc
# 한글 폰트 문제 해결
rc('font', family='AppleGothic')
# 차트 축 <- 음수 부호 지원
plt.rcParams['axes.unicode_minus'] = False

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error   import HTTPError
from urllib.error   import URLError

import requests

### 1. 페이지 요청
- requests 모듈
- url.request.urlopen
- 크롤링 시 두개의 방식의 큰 차이는 존재하지 않는다.
    - urlopen을 사용하면 error처리를 하기 졶은 장점이 있다

In [6]:
webpage = requests.get('https://www.daangn.com/hot_articles')
display(webpage)
webpage.text

<Response [200]>

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no">\n      <link rel="canonical" href="https://www.daangn.com/hot_articles" />\n\n  <title>당근마켓 중고거래 | 당신 근처의 당근마켓</title>\n<meta name="description" content="당근마켓에서 거래되는 인기 중고 매물을 소개합니다. 지금 당근마켓에서 거래되고 있는 다양한 매물을 구경해보세요." />\n<link rel="author" href="당근마켓" />\n<meta property="og:url" content="https://www.daangn.com/hot_articles" />\n<meta property="og:title" content="당근마켓 중고거래 | 당신 근처의 당근마켓" />\n<meta property="og:description" content="당근마켓에서 거래되는 인기 중고 매물을 소개합니다. 지금 당근마켓에서 거래되고 있는 다양한 매물을 구경해보세요." />\n<meta property="og:site_name" content="당근마켓" />\n<meta property="og:image" content="https://www.daangn.com/images/meta/home/flea_market.png" />\n<meta property="og:type" content="article" />\n<meta property="og:locale" content="ko_KR" />\n<meta property="fb:app_id" 

In [7]:
webpage = urlopen('https://www.daangn.com/hot_articles')
display(webpage)
webpage.read()

b'<!DOCTYPE html>\n<html lang="ko">\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no">\n      <link rel="canonical" href="https://www.daangn.com/hot_articles" />\n\n  <title>\xeb\x8b\xb9\xea\xb7\xbc\xeb\xa7\x88\xec\xbc\x93 \xec\xa4\x91\xea\xb3\xa0\xea\xb1\xb0\xeb\x9e\x98 | \xeb\x8b\xb9\xec\x8b\xa0 \xea\xb7\xbc\xec\xb2\x98\xec\x9d\x98 \xeb\x8b\xb9\xea\xb7\xbc\xeb\xa7\x88\xec\xbc\x93</title>\n<meta name="description" content="\xeb\x8b\xb9\xea\xb7\xbc\xeb\xa7\x88\xec\xbc\x93\xec\x97\x90\xec\x84\x9c \xea\xb1\xb0\xeb\x9e\x98\xeb\x90\x98\xeb\x8a\x94 \xec\x9d\xb8\xea\xb8\xb0 \xec\xa4\x91\xea\xb3\xa0 \xeb\xa7\xa4\xeb\xac\xbc\xec\x9d\x84 \xec\x86\x8c\xea\xb0\x9c\xed\x95\xa9\xeb\x8b\x88\xeb\x8b\xa4. \xec\xa7\x80\xea\xb8\x88 \xeb\x8b\xb9\xea\xb7\xbc\xeb\xa7\x88\xec\xbc\x93\xec\x97\x90\xec\x84\x9c \xea\xb1\xb0\xeb\x9e\x98\xeb\x90\x98\xea\xb3\xa0 \xec\x9e\x

### 2. html parsing (BeautifulSoup)
- url에서 요청을 받아온 데이터들을 읽어보면 전체의 데이터가 하나의 str로 담겨있는걸 알수있다 (urlopen의 경우 bytes타입이다)
- str형식으로는 html 문서를 효과적으로 처리하기가 불가능하다
- BeautifulSoup 라이브러리를 이용해 html문서 형식으로 parsing하면 보다 효율적으로 사용 가능

In [12]:
webpage = urlopen('https://www.daangn.com/hot_articles')
display(webpage)
BeautifulSoup(webpage.read(),'html.parser')

<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
<link href="https://www.daangn.com/hot_articles" rel="canonical"/>
<title>당근마켓 중고거래 | 당신 근처의 당근마켓</title>
<meta content="당근마켓에서 거래되는 인기 중고 매물을 소개합니다. 지금 당근마켓에서 거래되고 있는 다양한 매물을 구경해보세요." name="description">
<link href="당근마켓" rel="author"/>
<meta content="https://www.daangn.com/hot_articles" property="og:url">
<meta content="당근마켓 중고거래 | 당신 근처의 당근마켓" property="og:title">
<meta content="당근마켓에서 거래되는 인기 중고 매물을 소개합니다. 지금 당근마켓에서 거래되고 있는 다양한 매물을 구경해보세요." property="og:description"/>
<meta content="당근마켓" property="og:site_name"/>
<meta content="https://www.daangn.com/images/meta/home/flea_market.png" property="og:image"/>
<meta content="article" property="og:type"/>
<meta content="ko_KR" property="og:locale"/>
<meta content="1463621440622064" property="fb:app_id"/>
<meta content=

In [20]:
# soup의 타입을 확인해보자
# str -> bs4.BeautifulSoup 객체가 되었다
webpage = requests.get('https://www.daangn.com/hot_articles')
print(type(webpage.text))
soup = BeautifulSoup(webpage.text,'html.parser')
print(type(soup))

<class 'str'>
<class 'bs4.BeautifulSoup'>


### 3. 에러처리
- 없는 페이지에 request를 할 경우 requsets 라이브러리는 에러를 방생시키 않는다
- urllib은 error를 발생시키고 해당 에러또한 import 하여 잡아낼수 있다

In [16]:
try : 
    html = urlopen('https://pythondojang.bitbucket.io/weather/observation/currentweather.html')
    #print(html.read())
except HTTPError as he:
    print('http error')
except URLError as ue:
    print('url error')
else:
    soup = BeautifulSoup(html.read(),'html.parser')
    display(soup)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>도시별 현재날씨 &gt; 지상관측자료 &gt; 관측자료 &gt; 날씨 &gt; 기상청 </title>
<link href="http://www.kma.go.kr/favicon2.ico" rel="shortcut icon"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/share/css/import.css?20160530" rel="stylesheet" type="text/css"/>
<script src="/share/js/jquery-1.7.1.min.js" type="text/javascript"></script>
<script src="/share/js/common.js?ver=20150417" type="text/javascript"></script>
<link href="/share/css/warninglayer.css" rel="stylesheet" type="text/css"/>
<!--[if gte IE 7]><link rel="stylesheet" type="text/css" href="/share/css/ie7.css" /><![endif]-->
<meta content="기상청 " name="title"/>
<meta content="기상청" name="author"/>
<meta content="날씨, 기상청" name="keywords"/>
<meta content="서울시 기상청 사이트입니다." name="description"/>
<meta content="IE=Emul

### 4. BeautifulSoup 사용법
- tag를 통한 접근
- css 선택자를 통한 접근

#### tag를 통한 접근
- soup.tag, soup.find()함수 => 처음 등장하는 태그를 중심으로 가져옴
- soup.find_all()함수 => 해당 태그를 가진 모든 값(BeautifulSoup 객체)을 리스트 형식으로 가져옴
- 태그와 더불어 id class 속성을 함께 정의 가능하다

In [2]:
webpage = requests.get('https://www.daangn.com/hot_articles')
soup = BeautifulSoup(webpage.text,'html.parser')

In [5]:
soup.h1

<h1 id="fixed-bar-logo-title">
<a href="https://www.daangn.com/">
<span class="sr-only">당근마켓</span>
<img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
</a> </h1>

In [34]:
soup.find('h1')

<h1 id="fixed-bar-logo-title">
<a href="https://www.daangn.com/">
<span class="sr-only">당근마켓</span>
<img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
</a> </h1>

In [35]:
soup.find_all('h1')

[<h1 id="fixed-bar-logo-title">
 <a href="https://www.daangn.com/">
 <span class="sr-only">당근마켓</span>
 <img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
 </a> </h1>,
 <h1 class="head-title" id="hot-articles-head-title">
     
     
     중고거래 인기매물
   </h1>]

In [30]:
# find_all()사용시 타입을 확인해보자 
# <class 'bs4.element.ResultSet'>으로 나오지만 파이썬의 내장 리스트형식이라고 생각해도 무방하다
print(type(soup.find_all('h1')))
print()
for i in soup.find_all('h1'):
    print(i)

<class 'bs4.element.ResultSet'>

<h1 id="fixed-bar-logo-title">
<a href="https://www.daangn.com/">
<span class="sr-only">당근마켓</span>
<img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
</a> </h1>
<h1 class="head-title" id="hot-articles-head-title">
    
    
    중고거래 인기매물
  </h1>


In [53]:
# find_all()사용시 리스트로 태그를 넣어주면
# 동시에 해당하는 태그들을 다 찾는다
soup.find_all(['h1','p'])

[<h1 id="fixed-bar-logo-title">
 <a href="https://www.daangn.com/">
 <span class="sr-only">당근마켓</span>
 <img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
 </a> </h1>,
 <h1 class="head-title" id="hot-articles-head-title">
     
     
     중고거래 인기매물
   </h1>,
 <p>당근마켓 앱에서 따뜻한 거래를 직접 경험해보세요!</p>]

#### ⭐ node tracking

In [38]:
# node tracking
# 태그를 계층적으로 접근 가능하다
# 다만 반드시 반환 타입이 BeautifulSoup 객체인 경우에만 가능하다
soup.find('h1').find('a').find('span')

<span class="sr-only">당근마켓</span>

In [39]:
# 반환 타입이 Result set이기 때문에 에러발생
soup.find_all('h1').find('a')

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [41]:
# for문을 이용하면 ResultSet객체도 안에서 find 다시이용가능하다
for i in soup.find_all('h1'):
    display(i.find('a'))

<a href="https://www.daangn.com/">
<span class="sr-only">당근마켓</span>
<img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
</a>

None

#### ⭐ tag 와 속성을 함께 사용
- class id 다 가능하다
- {'id':'value'} {'class':'value'} 
- id='value' class_ = 'value'

In [44]:
display(soup.find_all('h1'))

[<h1 id="fixed-bar-logo-title">
 <a href="https://www.daangn.com/">
 <span class="sr-only">당근마켓</span>
 <img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
 </a> </h1>,
 <h1 class="head-title" id="hot-articles-head-title">
     
     
     중고거래 인기매물
   </h1>]

In [45]:
# 방법 1
# id에 해당하는 h1만 찾아왔다 
display(soup.find_all('h1',attrs={'id':'fixed-bar-logo-title'}))

[<h1 id="fixed-bar-logo-title">
 <a href="https://www.daangn.com/">
 <span class="sr-only">당근마켓</span>
 <img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
 </a> </h1>]

In [47]:
# 방법 2
# id에 해당하는 h1만 찾아왔다
display(soup.find_all('h1',id='fixed-bar-logo-title'))

[<h1 id="fixed-bar-logo-title">
 <a href="https://www.daangn.com/">
 <span class="sr-only">당근마켓</span>
 <img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
 </a> </h1>]

#### ⭐ 많이 하는 실수

In [49]:
soup.find('h1').find('h1') # 이미 h1태그를 찾은 상태에서 h1을 다시 찾으면 None을 반환한다

#### css 선택자를 통한 접근
- css 선택자가 편한 경우도 존재한다 => 특정 태그 혹은 속성 하위를 지칭하고 싶을때
- find_all 과 마찬가지로 ResultSet으로 반환한다
- id: #
- class: .
- direct 자손과 일반 자손은 구분된다

In [63]:
# 단순 태그 select
soup.select('h1')

[<h1 id="fixed-bar-logo-title">
 <a href="https://www.daangn.com/">
 <span class="sr-only">당근마켓</span>
 <img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
 </a> </h1>,
 <h1 class="head-title" id="hot-articles-head-title">
     
     
     중고거래 인기매물
   </h1>]

In [56]:
# class select
soup.select('card-title')

[<h2 class="card-title">Zotac gaming gtx1660 super 팝니다.</h2>,
 <h2 class="card-title">그래픽카드 PT-GTX1060</h2>,
 <h2 class="card-title">장인한과 파지약과</h2>,
 <h2 class="card-title">쌀10kg</h2>,
 <h2 class="card-title">컴퓨터 본체 팝니다.(GTX 1060 6G)</h2>,
 <h2 class="card-title">삼성냉동고</h2>,
 <h2 class="card-title">삼섬 27인치 QHD 커브드모니터 C27JG54(144Hz), 총 2대 저렴하게 판매해요.(모니터암 함께 드려요)</h2>,
 <h2 class="card-title">선반</h2>,
 <h2 class="card-title">직접캔 고구마</h2>,
 <h2 class="card-title">보이스캐디T8</h2>,
 <h2 class="card-title">신세계상품권 30-&gt;28</h2>,
 <h2 class="card-title">샤이니 앨범 (여러가지)</h2>,
 <h2 class="card-title">나이키 운동화 250mm (새상품) 에어포스1 울트라포스</h2>,
 <h2 class="card-title">삼성 LED TV 32인치 판매합니다! </h2>,
 <h2 class="card-title">직접잡은쭈꾸미</h2>,
 <h2 class="card-title">철제서랍장</h2>,
 <h2 class="card-title">장인한과(파지약과) 팝니다</h2>,
 <h2 class="card-title">누룽지 만드는 기계</h2>,
 <h2 class="card-title">3단선반 진열장 렌지대</h2>,
 <h2 class="card-title">LED티비 겸 모니터 팝니다(삼성)</h2>,
 <h2 class="card-title">🥕당근최저가🥕 [위닉스] 제습기</h2>,
 <h2 class="ca

In [57]:
# select id
soup.select('#content')

[<section id="content">
 <h1 class="head-title" id="hot-articles-head-title">
     
     
     중고거래 인기매물
   </h1>
 <nav id="hot-articles-navigation">
 <select class="hot-articles-nav-select" id="region1" name="region1" onchange="changeRegion('r1', this.value)"><option value="">지역을 선택하세요</option><option value="서울특별시">서울특별시</option>
 <option value="부산광역시">부산광역시</option>
 <option value="대구광역시">대구광역시</option>
 <option value="인천광역시">인천광역시</option>
 <option value="광주광역시">광주광역시</option>
 <option value="대전광역시">대전광역시</option>
 <option value="울산광역시">울산광역시</option>
 <option value="세종특별자치시">세종특별자치시</option>
 <option value="경기도">경기도</option>
 <option value="강원도">강원도</option>
 <option value="충청북도">충청북도</option>
 <option value="충청남도">충청남도</option>
 <option value="전라북도">전라북도</option>
 <option value="전라남도">전라남도</option>
 <option value="경상북도">경상북도</option>
 <option value="경상남도">경상남도</option>
 <option value="제주특별자치도">제주특별자치도</option>
 <option value="집현동">집현동</option></select>
 <select class="hot-articles

In [1]:
# direct child
soup.select('#fixed-bar > div')

NameError: name 'soup' is not defined

In [74]:
# 그냥 child
# 위의 direct와 구분 된다
soup.select('#fixed-bar  div')

[<div id="fixed-bar-wrap">
 <h1 id="fixed-bar-logo-title">
 <a href="https://www.daangn.com/">
 <span class="sr-only">당근마켓</span>
 <img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
 </a> </h1>
 <section id="fixed-bar-search">
 <div class="search-input-wrap">
 <span class="sr-only">검색</span>
 <input class="fixed-search-input" id="header-search-input" name="header-search-input" placeholder="동네 이름, 물품명 등을 검색해보세요!" type="text"/>
 <button id="header-search-button">
 <img alt="Search" class="fixed-search-icon" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/search-icon-7008edd4f9aaa32188f55e65258f1c1905d7a9d1a3ca2a07ae809b5535380f14.svg"/>
 </button>
 </div>
 </section>
 <section class="fixed-bar-menu">
 <div class="fixed-download-wrapper">
 <input class="fixed-checkbox" id="fixed-menu-checkbox" type="checkbox"/>
 <label class="fixed-label"

### 5. BeautifulSoup 속성값 ,텍스트 뽑아내기
- text는 soup.text
- 속성값은 ['property name']을 이용한다

In [95]:
soup.find('h1')

<h1 id="fixed-bar-logo-title">
<a href="https://www.daangn.com/">
<span class="sr-only">당근마켓</span>
<img alt="당근마켓" class="fixed-logo" src="https://d1unjqcospf8gs.cloudfront.net/assets/home/base/header/logo-basic-24b18257ac4ef693c02233bf21e9cb7ecbf43ebd8d5b40c24d99e14094a44c81.svg"/>
</a> </h1>

In [100]:
soup.select('h1 span')[0]

<span class="sr-only">당근마켓</span>

In [101]:
soup.select('h1 span')[0].text

'당근마켓'

In [102]:
soup.select('h1 span')[0]['class']

['sr-only']